In [8]:
import collections

import datasets
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import tqdm
print("torchtext version : " , torchtext.__version__)
print("torch version : ",torch.__version__)

torchtext version :  0.18.0+cpu
torch version :  2.3.0+cpu


In [9]:
train_data, test_data = datasets.load_dataset("imdb", split=["train", "test"])

In [10]:
train_data,test_data

(Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 }))

In [11]:
train_data.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [12]:
train_data[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [14]:
#pip install torchtext==0.4 resolve an issue when trying to import the module
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer("basic_english")


c:\Users\comet\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [29]:
#example of use of the tokenizer :

tokenizer("Hello world ! How are you doing today")

['hello', 'world', '!', 'how', 'are', 'you', 'doing', 'today']

In [30]:
def tokenize_example(example,tokenizer, max_length):
    tokens = tokenizer(example["text"])[:max_length]
    return {"tokens" : tokens}

In [31]:
max_length = 256

train_data = train_data.map(
    tokenize_example, fn_kwargs={"tokenizer" : tokenizer, "max_length" : max_length}
)

test_data = test_data.map(
    tokenize_example, fn_kwargs={"tokenizer" : tokenizer, "max_length" : max_length}
)

In [32]:
train_data

Dataset({
    features: ['text', 'label', 'tokens'],
    num_rows: 25000
})

In [33]:
train_data.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [34]:
train_data[0]["tokens"][:25]

['i',
 'rented',
 'i',
 'am',
 'curious-yellow',
 'from',
 'my',
 'video',
 'store',
 'because',
 'of',
 'all',
 'the',
 'controversy',
 'that',
 'surrounded',
 'it',
 'when',
 'it',
 'was',
 'first',
 'released',
 'in',
 '1967',
 '.']

In [35]:
test_size = 0.25

train_valid_data = train_data.train_test_split(test_size=test_size)
train_data = train_valid_data["train"]
valid_data = train_valid_data["test"]

In [36]:
len(train_data), len(valid_data), len(test_data)


(18750, 6250, 25000)

In [38]:
min_freq = 5
special_tokens = ["<unk>" , "<pad>"]
vocab = torchtext.vocab.build_vocab_from_iterator(train_data["tokens"],min_freq=min_freq,specials=special_tokens)

TypeError: build_vocab_from_iterator() got an unexpected keyword argument 'min_freq'